# 人脸识别

# 1. 实验介绍

## 1.1  实验内容

本实验采用特征脸（Eigenface）算法进行人脸识别。

特征脸（Eigenface）是第一种有效的人脸识别方法。在模型训练过程中，首先要根据测试数据求出平均脸，然后将前 K 个特征脸保存下来，利用这 K 个特征脸对测试人脸进行识别，此外对于任意给定的一张人脸图像，可以使用这K个特征脸对原图进行重建。

本次实验已经给出模型的代码，并准备了人脸数据集。你需要调整代码，完成多个实验（详见后面作业部分），并记录实验结果，回答问题。


## 1.2 实验要求

阅读并运行代码，了解特征脸（Eigenface）算法的实现过程，然后根据作业要求，更改代码，进行实验，并记录实验结果，回答问题。

## 1.3 参考文献

+ 特征脸，维基百科：https://zh.wikipedia.org/wiki/%E7%89%B9%E5%BE%81%E8%84%B8
+ Eigenface for Recognition： https://sites.cs.ucsb.edu/~mturk/Papers/jcn.pdf
+ The Yale Face Database： http://cvc.cs.yale.edu/cvc/projects/yalefaces/yalefaces.html

# 2.实验部分



## 2.1 试验数据介绍

本作业中使用的人脸图片来自 [Yale Face Database](http://cvc.yale.edu/projects/yalefacesB/yalefacesB.html)。整个数据集包含来自 10 名志愿者的 5760 张照片，每名志愿者的照片都包含 9 个不同的表情以及 64 种不同的光照条件。


本作业中，我们只使用其中 640 张正脸的照片。这些照片保存在 datasets/yaleBfaces-momodel 文件夹下。在左侧的文件列表中双击进入该文件夹，你将发现这些照片又被进一步的分为了 5 个不同的文件夹，其中 subset0 文件夹中的照片，拍摄时灯光是从正面照射的，所以整张照片都比较明亮，而其他的 4 个文件夹，从 subset1 到 subset4，灯光的照射角度越来越偏，所以照片中的阴影也越来越大。


在本次实验中，subset0 文件夹中的照片将作为训练集，而 subset1 到 subset4 这 4 个文件夹中的照片将作为测试集。

In [ ]:
# 定义数据集的目录路径
yaleBfaces_data_base_path = "./datasets/yalebfaces-momodel/yaleBfaces"

下面我们来展示一张数据集里的人脸照片看看。

In [ ]:
# 导入必要的包
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
from PIL import Image
import os

我们定义一个读取单张图片的函数和一个展示照片的函数

In [ ]:
def read_one_img(path):
    """
    根据路径读取一张人脸图片
    
    :param path: 图片的路径
    :return: 
    """
    # 图片路径
    # 以灰度模式读取图片
    img_sample = Image.open(path).convert('L')
    
    # 把图片格式转为 numpy array 格式
    img_sample = np.array(img_sample, 'uint8')
    
    return img_sample


def show_img(img):
    """
    展示一张图片
    
    :param img: numpy array 格式的图片
    :return: 
    """
    # 展示图片
    plt.imshow(img, 'gray')
    plt.axis('off')
    plt.show()

使用上面定义的函数，查看照片

In [ ]:
#输入照片的路径进行查看
img_sample = read_one_img(os.path.join(yaleBfaces_data_base_path, "subset0/person01_01.png"))
show_img(img_sample)


## 2.2 获取训练数据

我们定义 `get_images()` 函数，其输入是照片存放的文件夹路径，该函数将读取输入的文件夹路径下的所有照片，将其转为 1 维，统一保存到一个矩阵中，然依据图片名提取标签，最终，该函数将输出这个照片矩阵及其中每张照片的标签。

In [ ]:
def get_images(path):
    """
    读取输入的文件夹路径下的所有照片，读取输入的文件夹路径下的所有照片，将其转为 1 维，
    统一保存到一个矩阵中，然依据图片名提取标签，最终该函数将输出这个照片矩阵及其中每
    张照片的标签。
    
    :param path: 照片存放的文件夹路径
    :return: numpy matrix 格式的处理好的图片，及 list 格式的各个图片的标签
    """
    # 首先获取所有人脸图片的路径
    image_paths = [os.path.join(path, f) for f in os.listdir(path) if
                   f.endswith('png')]
    
    # 所有的人脸区域图片都将保存在 images 这个矩阵中
    images = np.mat(np.zeros((len(image_paths), 2500)))
    
    trainset_labels = []
    
    # 对于每一张图片
    for index, image_path in enumerate(image_paths):
        
        # 读取图片并将其转为灰度图
        image_pil = Image.open(image_path).convert('L')
        
        # 把图片转为 numpy array 格式
        image = np.array(image_pil, 'uint8')
        
        # 把 2 维的平面图像转为 1 维
        img_1D = image.flatten()
        
        # 把处理后的图片保存到 images 中
        images[index, :] = img_1D
        
        # 提取图片名作为图片的标签
        trainset_labels.append(image_path.split('/')[-1][6:8])
        
    # 得到最终处理好的人脸图片和各个图片的标签
    return images.T, trainset_labels

我们从 subset0 文件夹下的取得全部 70 张人脸图片及其标签作为训练数据

In [ ]:
# 调用前面定义好的 get_images 方法得到处理好的人脸数据
trainset, trainset_labels =  get_images(os.path.join(yaleBfaces_data_base_path, "subset0"))

我们可以看一下处理后的第一张图片的 shape 及其标签， 注意 shape[0] 等于 2500， 是因为我们将每一张图片是 50 * 50 的，我们将其从 2 维转为了 1 维。

In [ ]:
trainset[:,1].shape

In [ ]:
trainset_labels[0]

## 2.3 训练模型

下面给出了训练特征脸（Eigenface）算法的代码，其输入是人脸数据训练集，希望提取的主特征数，算法先根据测试数据求出平均脸，然后计算训练数据里每张脸与平均脸的差异，求差异矩阵的特征值和特征向量，取前 K 个特征向量，计算出 K 张特征脸，然后就可以利用这 K 个特征脸对测试人脸进行识别了。


具体算法细节，请查看上面给出的参考文献。

In [ ]:
def eigen_train(trainset, K=10):
    """
    训练特征脸（eigenface）算法的实现
    
    :param trainset: 使用 get_images 函数得到的处理好的人脸数据训练集
    :param K: 希望提取的主特征数
    :return: 训练数据的平均脸, 特征脸向量, 和使用特征脸向量表示的训练数据
    """
    # 使用 numpy 的 mean 方法得到平均脸
    avg_img = np.mean(trainset, 1)
    
    # 得到每张脸与平均脸的差异的矩阵
    diff = trainset - avg_img

    # 使用 np.linalg.eig 方法得到差异矩阵的 特征值 eig_vals 以及 特征向量 eig_vects
    eig_vals, eig_vects = np.linalg.eig(
        np.mat(diff.T * diff))

    # 得到特征脸向量并 normalize
    face_vects = diff * eig_vects
    l2norm = np.sqrt((face_vects.T * face_vects).diagonal())
    face_vects = face_vects/l2norm
    
    # 得到 topK 特征向量
    face_vects = face_vects[:, 0:K]
    
    # 得到使用特征脸向量表示的训练数据
    trainset_vects = face_vects.T * diff 
    
    # 训练数据的平均脸, 特征脸向量, 和使用特征脸向量表示的训练数据
    return avg_img, face_vects, trainset_vects

In [ ]:
avg_img, face_vects, trainset_vects = eigen_train(trainset, K=10)

我们把平均脸的图片还原回 2 维，然后展示出来看看平均脸的样子

In [ ]:
# 把平均脸还原为 2 维
IMAGE_SIZE = (50,50)
avg_img_reshape = avg_img.reshape(IMAGE_SIZE)

# 展示还原后的图片
show_img(avg_img_reshape)

我们把一张特征脸还原回 2 维，然后展示出来看看

In [ ]:
show_img(face_vects[:, 0].reshape(50, 50))

## 2.4 模型测试

接下来，我们使用上面得到的模型在测试集上进行测试。

首先读取 subset1 中的人脸数据作为测试集

In [ ]:
# 调用前面定义好的 get_images 方法得到处理好的人脸数据
testset, testset_labels =  get_images(os.path.join(yaleBfaces_data_base_path, "subset1"))

然后使用特征脸（Eigenface）算法对 subset1 中的人脸照片进行预测，我们在这里定义了 eigenTest 函数，其输入是使用特征脸向量表示的训练数据，训练集的平均人脸数据，特征脸向量，训练集的标签数据，测试数据和测试数据的标签。

In [ ]:
def eigen_test(trainset_vects, avg_img, face_vects, trainset_labels,
               testset, testset_labels):
    """
    用特征脸（eigenface）算法对输入的测试集进行预测，并打印出预测正确数和总测试样本数
    
    :param trainset_vects: 使用特征脸向量表示的训练数据
    :param avg_img: 训练集的平均人脸数据
    :param face_vects: 特征脸向量
    :param trainset_labels: 训练集的标签数据
    :param testset: 测试数据
    :param testset_labels: 测试数据的标签
    :return: None
    """
    # 计算测试数据和训练数据平均脸的差异
    diff_test_avg = testset - avg_img
    
    # 得到使用特征脸向量表示的测试数据
    test_vec = face_vects.T * diff_test_avg
    
    # 初始化模型预测的正确数为 0
    right_answer_number = 0
    
    for i in range(test_vec.shape[1]):
        # 对每一张测试图片，计算其特征脸向量表示与每一张训练图片的特征脸向量表示的欧式距离，
        # 得到与其最相似的图片，并将最相似照片的标签作为它的预测标签
        x = np.square(trainset_vects - test_vec[:, i]).sum(axis=0).argmin()
        predict_label = trainset_labels[x]
        
        # 如果标签与真实标签相同，则模型预测的正确数 +1
        if predict_label == testset_labels[i]:
            right_answer_number += 1
            
    # 查看模型预测正确数和总的测试照片数
    print('预测正确数：', right_answer_number)
    print('总测试样本数：', len(testset_labels))


In [ ]:
eigen_test(trainset_vects, avg_img, face_vects, trainset_labels, testset,
           testset_labels)

## 2.5 使用特征向量重建图片

特征脸（Eigenface）算法不仅可以用来进行人脸识别，对于任意给定的一张人脸照片，也可以使用得到的特征脸对原图进行重建。

我们使用上面从 subset0 中读取的那一张照片，我们再看一下原始的样子

In [ ]:
# 展示改图片的原始的样子
show_img(img_sample)

接下来，我们使用特征脸来表征原始照片，先将原始照片从 2 维转为 1 维，然后计算该样本照片与平均脸的差异，再得到使用特征脸向量表示的样本照片

In [ ]:
# 先把照片从 2 维转为 1 维
img_sample_1D = img_sample.reshape(2500, 1)

# 计算该样本照片与平均脸的差异
diff_sample_avg = img_sample_1D - avg_img

# 得到使用特征脸向量表示的样本照片
sample_vec = face_vects.T * diff_sample_avg

我们来看看 sample_vec 的 shape

In [ ]:
sample_vec.shape

我们看到使用特征脸向量来表示原始图片，将原始图片的存储空间由 2500 降低到了 10，这就实现了图片压缩存储的目的。


接下来，我们进行图片的恢复。

In [ ]:
# 恢复原始图片，并将其从 1 维转为 2 维
recovered_sample_img = (face_vects * sample_vec + avg_img).reshape(50, 50)

In [ ]:
# 展示恢复后的图片
show_img(recovered_sample_img)

我们看到，图片在一定程度上进行了恢复，但很多细节丢失了。

# 3. 问题

### 问题 1:  观察主特征数量的变化是否会影响模型表现(20分)

使用不同数量（5 个，10个，20个）的主特征，来进行模型测试，观察随着主特征数量的增加，模型的表现是否会有变化。
在下方区域写下你的实验结果，并对这一现象进行解释。


=================== 答案 1 ======================

双击此区域开始编写第 1 题的答案，请勿在别的位置作答


==============================================

### 问题2:  观察剔除最主要的三个主特征后，使用剩余的特征进行模型预测，模型的表现如何(20分)

剔除最主要的三个主特征后，使用剩余的特征来进行模型测试，观察模型的表现是否会有变化。 在下方区域写下你的实验结果，并对这一现象进行解释。


=================== 答案 2 ======================

双击此区域开始编写第 2 题的答案，请勿在别的位置作答


==============================================

### 问题 3:  观察照片中光照的变化是否会影响模型表现(20分)

我们知道从 subset1 到 subset4，照片中灯光的角度越来约偏，图片中的阴影也越来越大。那么光照的不同会对模型有影响吗？更改代码，在这 4 个测试集上测试模型，观察模型表现。在下方区域写下你的实验结果，并对这一现象进行解释。


=================== 答案 3 ======================

双击此区域开始编写第 3 题的答案，请勿在别的位置作答


==============================================

### 问题 4:  分别使用 top 3, top5, top 10 主特征重建照片，观察生成的图片(20分)

从 subset0 中的挑选一张照片，然后分别使用 top 3, top5, top 10 主特征重建这张图片，观察重建后的图片与原图片的差异。把你的观察结果写在下面。


=================== 答案 4 ======================

双击此区域开始编写第 4 题的答案，请勿在别的位置作答


==============================================

### 问题 5:  调查人脸识别技术的发展(20分)

做一些人脸识别技术发展情况（最新的技术、应用的场景等）的调查，把你的调查结果写在下面。


=================== 答案 5 ======================

双击此区域开始编写第 5 题的答案，请勿在别的位置作答


==============================================